In [4]:
# The code below computes the HAC standard errors of the regression coefficients using the Hansen and Hodrick (1980) method
# Python has a package called statsmodels that can compute the HAC standard errors (Newey West) of the regression coefficients
# This code supplements the method of Hansen and Hodrick (1980) method.
# The regression is Y = a + bX + e, where Y is the postretrun and X is the prereturn
# The residuals are followed MA(k-1) process
import numpy as np
import pandas as pd
import statsmodels.api as sm  # Need to install (pip install statsmodels)

def compute_hansen_hodrick_covariance(model, X, K):
    T, p = X.shape
    residuals = model.resid.values.reshape(-1, 1)
    XtX = np.dot(X.T, X) / T
    S_0 = np.zeros((p, p))
    
    for t in range(T):
        epsilon_sq = residuals[t] ** 2
        x_t = X[t].reshape(-1, 1)
        S_0 += epsilon_sq * np.dot(x_t, x_t.T)
    S_0 /= T
    
    for j in range(1, K):
        for t in range(j, T):
            epsilon = residuals[t]
            epsilon_j = residuals[t - j]
            x_t = X[t].reshape(-1, 1)
            x_t_j = X[t - j].reshape(-1, 1)
            S_0 += (1 / T) * (epsilon * epsilon_j * (np.dot(x_t, x_t_j.T) + np.dot(x_t_j, x_t.T)))
    
    S_inv = np.linalg.inv(S_0)
    hac_hh = np.linalg.inv(XtX @ S_inv @ XtX) / T
    
    return hac_hh

# Import the data, replace your code with your own data path
# Get the residuals of the regression Y on X, where Y is the postretrun and X is the prereturn
# The residuals are followed MA(k-1) process
# Calculate the HAC covariance matrix of the residuals using two different methods: Hansen and Hodrick (1980) and Newey and West (1987)

data = pd.read_excel('Assignment2Data.xlsx', sheet_name='FamaFrenchSize')
data['Small'] /= 100
data['Medium'] /= 100
data['Big'] /= 100
data['r_small'] = np.log(data['Small'] + 1)
data['r_medium'] = np.log(data['Medium'] + 1)
data['r_big'] = np.log(data['Big'] + 1)

Paramaters = {'Coefficient': [], 'Hansen and Hodrick': [], 'Newey and West': [], 'Usual se': []}
# First, using small portfolio as sample for example.

K = [1, 12, 24, 36, 48, 60, 72, 96, 120] # The window size
for k in K:
    data['Pre_return'] = data['r_small'].rolling(window=k).sum()
    data['Post_return'] = data['r_small'].shift(-k).rolling(window=k).sum()
    sample = data.dropna(subset=['Pre_return', 'Post_return'])
    X = sm.add_constant(sample['Pre_return'])
    y = sample['Post_return']
    model = sm.OLS(y, X).fit()
    
    hac_hh = compute_hansen_hodrick_covariance(model, X.values, k-1)
    stand_error = np.sqrt(np.diag(hac_hh))
    
    Paramaters['Coefficient'].append(model.params.iloc[1])
    Paramaters['Hansen and Hodrick'].append(stand_error[1])
    Paramaters['Usual se'].append(model.bse.iloc[1])
    nw_se = model.get_robustcov_results(cov_type='HAC', maxlags=k-1).bse[1]
    Paramaters['Newey and West'].append(nw_se)
Paramaters=pd.DataFrame(Paramaters)
print(Paramaters)


   Coefficient  Hansen and Hodrick  Newey and West  Usual se
0     0.176569            0.058642        0.058642  0.028844
1    -0.029686            0.127908        0.107127  0.029519
2    -0.133088            0.098425        0.130145  0.029578
3    -0.261167            0.055644        0.081523  0.025294
4    -0.426142            0.068754        0.073599  0.020926
5    -0.452402            0.070975        0.087591  0.021227
6    -0.307614            0.132483        0.127692  0.024342
7    -0.316318            0.180330        0.154566  0.026594
8    -0.305459            0.191501        0.151243  0.026230
